# Curricular Analytics - What if? Simulator

Hi! This notebook is here to walk you through simulating changes to a curriculum. You give us the file and the desired change and we'll tell you how that affects complexity and centrality, and why. But first, let us do some basic-set up. Run the cell below to do that:

In [2]:
using CurricularAnalytics, CurricularAnalyticsDiff

Ok! We're all set! There are four modifications you can make: you can add a course, remove a course, add a prereq and remove a prereq. After adding the curriculum you wish to change by running the next cell, just scroll on down to the section you want to work with. Currently, we'll be adding the FILENAME file in the targets folder. Feel free to upload your own curriculum file and change the file path.

In [6]:
curr = read_csv("./targets/SY-Degree Plan-Marshall-CE25 2017.csv");
# dont' worry about this, this is just to make sure we're working with curricula, not degree plans
if typeof(curr) == DegreePlan
    curr = curr.curriculum
end

DegreePlan

# Make modifications

## Add a Course:


Let's add a course. A course is defined by a few things. It has a name, credit_hours, and prerequisites. It also usually has a list of classes that depend on it. i.e. the classes that have it as a prerequisite. Lets define those things right now. 

NOTE:
When editing the prereqs field, make sure the "COURSE NAME #" field is replaced by the name of the existing course in the target curriculum that is to become a prereq of your new course. The same concept applies to the "COURSE NAME #" in the dependencies field. Once you're done adding prerequisites and dependencies, delete the extra fields.
Pre refers to the type of prerequisite. For now, let's leave that alone.

As an example, let's try to add MATH 20C to a curriculum.

new_course_name = "MATH 20C"

new_course_credit_hours = 4.0

prereqs = Dict("MATH 18" => pre,
               "MATH 20B" => pre)
               
dependencies = Dict("MATH 20D" => pre, "MATH 20E" => pre)

Note that in a curriculum where MATH 20B is called "MATH 20B/10B" or something like that you must use the name used in the curriculum file, otherwise this will NOT work. 

In [ ]:
new_course_name = "COURSE NAME"
new_course_credit_hours = 4.0 # defualt, you can change it to 1.0,2.0,3.0, etc
prereqs = Dict("COURSE NAME 1" => pre,
               "COURSE NAME 2" => pre,
               "COURSE NAME 3" => pre)
dependencies = Dict("COURSE NAME 1" => pre,
                    "COURSE NAME 2" => pre,
                    "COURSE NAME 3" => pre)

Run the above cell once you're done, then run this one:

In [ ]:
new_curric = add_course(curr, new_course_name, new_course_credit_hours, prereqs, dependencies)

## Remove a course

Removing a course is pretty easy. Just tell us the name of the course to be removed in the field below. Remember, it must match exactly with the name used in the curriculum file. 

In [ ]:
course_to_remove = "COURSE NAME"

Then, run this cell:

In [ ]:
new_curric = remove_course(curr, course_to_remove)

## Add a prerequisite

To add a prerequisite to a course tell us the name of the course, and the name of the prerequisite you wish to add to it. Remember, both names must already exist in the curriculum and must match exactly with your names. For example, if MATH 20B shows up as "MATH 20B/10B" in your curriculum, use that.

In [ ]:
course_name = "COURSE TO GET A PREREQUISITE"
prerequisite = "PREREQ NAME"
req_type = pre

Once you're ready, run this cell:

In [ ]:
new_curric = add_prereq(curr, course_name, prerequisite,req_type)

## Remove a prerequisite

Removing a prerequisite needs the name of the course in question, and the name of the course which we want to remove as its prerequisite. Remember the names need to match the ones used in the curriculum file exactly.

In [ ]:
course_name = "COURSE TO LOSE A PREREQUISITE";
prerequisite = "PREREQUISITE TO BE LOST";

All good? Run the cell below

In [ ]:
new_curric = remove_prereq(curr,course_name,prerequisite);

# How did that affect the Curricular Metrics?

This is an easy question to answer. Just run the cell below after having made a new_curric with one of the previous modifications

In [ ]:
results = curricular_diff(curr,new_curric);

Once you're done with that, choose how you want to see your results: executive summary or pretty print

In [ ]:
executive_summary_curriculum(results)

In [ ]:
pretty_print_curriculum_results(results, ALL)

# Wrapping up

You can play around with this file, and you can write your new curriculum to file with the following cell. Unfortunately, we can't visualize the curricula here in this notebook, but you can take them over to the curricular analytics website to view.

In [1]:
write_csv(new_curric, "./newfile.csv")

The full package is available to you now to use. If you wish to make more complex modifications, be our guest! With these very basic building blocks you can make very complex edits to curricula.